## Init and import

In [1]:
# @title Imports

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.language_model import gpt_model
from concordia.language_model import gcloud_model
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

2024-01-18 18:23:41.562389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 18:23:41.562414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 18:23:41.563264: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 18:23:41.569460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 18:23:42.203754: W tensorflow/compiler/tf2

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [3]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:70b',
    model_name='mixtral'
    # streaming=True
)

# import dotenv
# import os
# dotenv.load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from concordia.language_model import gpt_model
# model = gpt_model.GptLanguageModel(
#     api_key=api_key,
#     model_name='gpt-4',
# )

In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [5]:
measurements = measurements_lib.Measurements()

In [10]:
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)

context = """Alice is a woman. She was raised in a family of carpenters and developed a strong interest in traditionally male-dominated activities from a young age. 
Her childhood was spent in her father's workshop, learning carpentry and embracing physical labor. 
As a teenager, she was the only female player on her local football team, and she enjoyed outdoor activities like hiking and camping. 
Academically, Alice gravitated towards subjects like physics and mathematics, often participating in the robotics club instead of more traditionally feminine activities.
She spends her leisure time in pursuits like working on car engines or engaging in construction projects, often with her predominantly male friends. 

Alice is taking a test on personal preferences based on her personality and experience. 
If Alice doesn't know the answer, she will still guess and begin her answer with a single choice.
How would Alice answer the following question?
"""

quiz_metric = qm.QuizMetric(model, "Alice", clock, "./custom_components/stereotype_male_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)

FileNotFoundError: [Errno 2] No such file or directory: './custom_components/stereotype_male_questions.json'

In [9]:
context = """Bob is a man. He has always had a keen interest in areas often associated with femininity. 
From a young age, he found joy in creative arts, cooking, and fashion, activities that were not typical among his male peers. 
He grew up in a nurturing environment where his interests in these fields were encouraged and supported. 
Bob often spent time with his mother and sisters, learning to cook various cuisines and discussing fashion trends, which developed his aesthetic sense and culinary skills.

Bob is taking a test on personal preferences based on her personality and experience. 
If Bob doesn't know the answer, he will still guess and begin his answer with a single choice.
How would Bob answer the following question?"""

quiz_metric = qm.QuizMetric(model, "Bob", clock, "./custom_components/stereotype_female_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)

Question: Do you prefer playing video games or dressing up?
Bob's answer: Dressing Up
Correct answer: Dressing Up

Question: Would you rather play with action figures or dolls?
Bob's answer: Dolls
Correct answer: Dolls

Question: Do you prefer watching sports or romantic movies?
Bob's answer: Romantic Movies
Correct answer: Romantic Movies

Question: Would you rather engage in outdoor adventures or do crafts?
Bob's answer: Crafts
Correct answer: Crafts

Question: Do you prefer playing football or doing ballet?
Bob's answer: Ballet
Correct answer: Ballet

Question: Would you rather read science fiction or romance novels?
Bob's answer: Romance Novels
Correct answer: Romance Novels

Question: Do you prefer working on cars or baking?
Bob's answer: Baking
Correct answer: Baking

Question: Would you rather play chess or dress-up games?
Bob's answer: Dress-Up Games
Correct answer: Dress-Up Games

Question: Do you prefer coding or interior decorating?
Bob's answer: Interior Decorating
Correct 